<a href="https://colab.research.google.com/github/Dhaneshkp/NLP/blob/main/Langchain_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
from langchain_astradb import AstraDBVectorStore
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from dotenv import load_dotenv
import os
import pandas as pd
#from google.ai import gemini
import pandas as pd
from langchain_core.documents import Document
from google.colab import userdata
from langchain.embeddings.openai import OpenAIEmbeddings

def dataconveter():
    product_data=pd.read_csv("flipkart_product_review.csv")

    data=product_data[["product_title","review"]]

    product_list = []

    # Iterate over the rows of the DataFrame
    for index, row in data.iterrows():
        # Construct an object with 'product_name' and 'review' attributes
        obj = {
                'product_name': row['product_title'],
                'review': row['review']
            }
        # Append the object to the list
        product_list.append(obj)



    docs = []
    for entry in product_list:
        metadata = {"product_name": entry['product_name']}
        doc = Document(page_content=entry['review'], metadata=metadata)
        docs.append(doc)
    return docs



In [8]:

load_dotenv()

GOOGLE_API_KEY=userdata.get("GOOGLE_API_KEY")
ASTRA_DB_API_ENDPOINT=userdata.get("ASTRA_DB_API_ENDPOINT")
ASTRA_DB_APPLICATION_TOKEN=userdata.get("ASTRA_DB_APPLICATION_TOKEN")
ASTRA_DB_KEYSPACE=userdata.get("ASTRA_DB_KEYSPACE")
YOUR_OPENAI_API_KEY=userdata.get("OPENAI_API_KEY")

In [ ]:
#!pip install google-cloud-aiplatform langchain

In [9]:
from langchain.embeddings import HuggingFaceEmbeddings
from transformers import AutoTokenizer, AutoModel

    # Choose your model

model_name = "sentence-transformers/all-MiniLM-L6-v2"

    # Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

    # Create HuggingFaceEmbeddings object
embedding = HuggingFaceEmbeddings(  model_name=model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

...

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
#embedding = OpenAIEmbeddings(openai_api_key='YOUR_OPENAI_API_KEY')
#embedding=GoogleGenerativeAIEmbeddings(google_api_key=GOOGLE_API_KEY,model='models/embedding-gecko-001')

In [13]:
from langchain.vectorstores import FAISS
#from google.ai import gemini


def ingestdata(status):


    storage=status

    if storage==None:
        docs=dataconveter()
        db = FAISS.from_documents(docs, embedding)
    else:
        return db
    return db

db=ingestdata(None)

docs = db.similarity_search("can you tell me the best bluetooth buds.")
for doc in docs:
    print(f"{doc.page_content} \n\nMetadata: {doc.metadata}")





Best earphone 

Metadata: {'product_name': 'OnePlus Bullets Wireless Z Bass Edition Bluetooth Headset'}
Best budget Bluetooth earphones by one plus ....sound quality superb.....bass also good .... vry good built quality ....fix 15 hours play back ....very fast order delivery by flipkart ....over all good 

Metadata: {'product_name': 'OnePlus Bullets Wireless Z Bluetooth Headset'}
Best bluetooth headset so far. I have used JBL, Boat, Portronics. This one is lightweight, sounds perfect and the charge lasts for days. The best part is that it has an amazing mic, unlike any other BT headset I used in past. 

Metadata: {'product_name': 'realme Buds Wireless Bluetooth Headset'}
Best Quality buds 

Metadata: {'product_name': 'realme Buds Q Bluetooth Headset'}


In [11]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

# Load tokenizer and model
model_name = "sentence-transformers/all-MiniLM-L6-v2"#"facebook/bart-base"  # Replace with your desired Bart model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Create HuggingFaceEmbeddings object
embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Assuming 'db' is your FAISS database created with the above embedding
retriever = db.as_retriever(search_kwargs={"k": 3})

PRODUCT_BOT_TEMPLATE = """
Your ecommercebot bot is an expert in product recommendations and customer queries.
It analyzes product titles and reviews to provide accurate and helpful responses.
Ensure your answers are relevant to the product context and refrain from straying off-topic.
Your responses should be concise and informative.

CONTEXT:
{context}

QUESTION: {question}

YOUR ANSWER:

"""

prompt = ChatPromptTemplate.from_template(PRODUCT_BOT_TEMPLATE)

# Create a text generation pipeline using BartForCausalLM
generator = pipeline(
    "text2text-generation",  # Use text2text-generation for Bart
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=50  # Adjust as needed
)

chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    # Extract the 'content' string from the HumanMessage object
    | (lambda x: x.to_messages()[0].content)
    | generator
    |(lambda x: x[0]['generated_text'])
    | StrOutputParser()
)

print(chain.invoke("can you tell me the best bluetooth buds?"))

If you want to use `BertLMHeadModel` as a standalone, add `is_decoder=True.`
Some weights of BertLMHeadModel were not initialized from the model checkpoint at sentence-transformers/all-MiniLM-L6-v2 and are newly initialized: ['cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
The model 'BertLMHeadModel' is not supported for text2text-generation. Supported models are ['BartForConditionalGeneration', 'BigBirdPegasusForConditionalGeneration', 'BlenderbotForConditionalGeneration', 'BlenderbotSmallForConditionalGeneration', 'EncoderDecoderModel', 'FSMTForConditionalGeneration', 'GPTSanJapaneseForConditionalGeneration', 'LEDForConditionalGeneration', 'LongT5ForConditionalGeneration', 'M2M100ForConditionalGene

your ecommercebot bot is an expert in product recommendations and customer queries. it analyzes product titles and reviews to provide accurate and helpful responses. ensure your answers are relevant to the product context and refrain from straying off - topic. your responses should be concise and informative. context : [ document ( metadata = { ' product _ name ' : ' oneplus bullets wireless z bass edition bluetooth headset ' }, page _ content = ' best earphone ' ), document ( metadata = { ' product _ name ' : ' oneplus bullets wireless z bluetooth headset ' }, page _ content = ' best budget bluetooth earphones by one plus.... sound quality superb..... bass also good.... vry good built quality.... fix 15 hours play back.... very fast order delivery by flipkart.... over all good ' ), document ( metadata = { ' product _ name ' : ' realme buds q bluetooth headset ' }, page _ content = ' best quality buds ' ) ] question : can you tell me the best bluetooth buds? your answer : medicalrb ear

In [15]:
prompt

ChatPromptTemplate(input_variables=['context', 'question'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template='\nYour ecommercebot bot is an expert in product recommendations and customer queries.\nIt analyzes product titles and reviews to provide accurate and helpful responses.\nEnsure your answers are relevant to the product context and refrain from straying off-topic.\nYour responses should be concise and informative.\n\nCONTEXT:\n{context}\n\nQUESTION: {question}\n\nYOUR ANSWER:\n\n'))])

In [18]:
# Install the necessary packages for Llama 2
!pip install -q transformers accelerate bitsandbytes

# Import the necessary modules from transformers
from transformers import AutoTokenizer
from transformers import AutoModelForCausalLM

# Load the quantized Llama 2 model
model_id = "gpt2"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id)

# Assuming 'db' is your FAISS database created with the above embedding
retriever = db.as_retriever(search_kwargs={"k": 3})

PRODUCT_BOT_TEMPLATE = """
Your ecommercebot bot is an expert in product recommendations and customer queries.
It analyzes product titles and reviews to provide accurate and helpful responses.
Ensure your answers are relevant to the product context and refrain from straying off-topic.
Your responses should be concise and informative.

CONTEXT:
{context}

QUESTION: {question}

YOUR ANSWER:

"""

prompt = ChatPromptTemplate.from_template(PRODUCT_BOT_TEMPLATE)

from langchain import HuggingFacePipeline
from transformers import pipeline

# Create a text generation pipeline using Llama 2
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=50,
    temperature=0.5
)

llm = HuggingFacePipeline(pipeline=pipe)

chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    # Extract the 'content' string from the HumanMessage object
    | (lambda x: x.to_messages()[0].content)
    | llm
    | StrOutputParser()
)

print(chain.invoke("can you tell me the best bluetooth buds?"))


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



Your ecommercebot bot is an expert in product recommendations and customer queries.
It analyzes product titles and reviews to provide accurate and helpful responses.
Ensure your answers are relevant to the product context and refrain from straying off-topic.
Your responses should be concise and informative.

CONTEXT:
[Document(metadata={'product_name': 'OnePlus Bullets Wireless Z Bass Edition Bluetooth Headset'}, page_content='Best earphone'), Document(metadata={'product_name': 'OnePlus Bullets Wireless Z Bluetooth Headset'}, page_content='Best budget Bluetooth earphones by one plus ....sound quality superb.....bass also good .... vry good built quality ....fix 15 hours play back ....very fast order delivery by flipkart ....over all good'), Document(metadata={'product_name': 'realme Buds Q Bluetooth Headset'}, page_content='Best Quality buds')]

QUESTION: can you tell me the best bluetooth buds?

YOUR ANSWER:


You can find a list of the best bluetooth buds here.

The best bluetooth b

In [6]:
!pip install sentence-transformers

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [3]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 9.8 MB/s eta 0:00:00


In [2]:
!pip install  langchain-astradb langchain langchain-openai datasets pypdf python-dotenv flask langchain_google_genai

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.3/990.3 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.1/47.1 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 27.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.5/156.5 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.1/316.1 kB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 377.3/377.3 kB 23.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.0/337.0 kB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 11.4 MB/s eta 0:

In [4]:
!pip install openai tiktoken

In [5]:
!pip install langchain_community
!pip install transformers
!pip install langchain

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 2.2 MB/s eta 0:00:00
